In [1]:
import json

In [15]:
if __name__ == "__main__":
    file_path = "Data_with_normalized_allergy_info.json"  # Replace with your path
    unique_allergies = extract_unique_allergies(file_path)

    print("Unique allergy_info entries:")
    for allergy in unique_allergies:
        print("-", allergy)

Unique allergy_info entries:
- Dairy
- Egg
- Fish and Shellfish
- Gluten
- May contain Wheat, Soy, Milk
- Mustard
- Nuts
- OATS
- Others
- Peanuts
- Sesame
- Soya
- Sulphites


In [17]:
import json

def extract_unique_allergies(json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    unique_allergies = set()

    for item in data:
        allergy_info = item.get("allergy_info")
        if isinstance(allergy_info, list):
            unique_allergies.update(allergy_info)
        elif isinstance(allergy_info, str):
            unique_allergies.add(allergy_info)

    return sorted(unique_allergies)


if __name__ == "__main__":
    file_path = "Data_with_normalized_allergy_info.json"  # Replace with your path
    unique_allergies = extract_unique_allergies(file_path)

    print("Unique allergy_info entries:")
    for allergy in unique_allergies:
        print("-", allergy)


Unique allergy_info entries:
- Dairy
- Egg
- Fish and Shellfish
- Gluten
- Mustard
- Nuts
- OATS
- Others
- Peanuts
- Sesame
- Soy
- Soya
- Sulphites
- Whear
- Wheat


In [3]:
from collections import defaultdict

In [4]:
# Define group keywords
category_keywords = {
    "Dairy": ["dairy", "milk", "cheese", "butter", "cream", "yogurt", "whey", "paneer", "ghee", "lactose"],
    "Gluten": ["gluten", "wheat", "barley", "semolina", "oat", "spelt", "breadcrumbs"],
    "Nuts": ["nut", "nuts", "almond", "peanut", "cashew", "pistachio", "hazelnut", "groundnut", "badam"],
    "Soya": ["soya", "soy", "lecithin", "emulsifier"],
    "Sesame": ["sesame", "tahini"],
    "Mustard": ["mustard"],
    "Sulphites": ["sulphite", "sulphites", "candied fruit", "dried fruits", "tutti frutti"],
    "Egg": ["egg"],
    "Fish/Shellfish": ["fish", "shellfish", "crustacean", "crab", "prawn"],
    "Coconut": ["coconut"],
    "Celery": ["celery"],
    "Phenylalanine": ["phenylalanine"],
    "Ragi": ["ragi"],
    "Oats": ["oats"],
}

In [6]:
allergy_entries = [
    "BARLEY", "CELERY", "Celery", "Coconut", "Coconut (if used)", "Crustaceans (Crab)",
    "Crustaceans (Prawns)", "Dairy (Butter)", "Dairy (Butter, Cream)", "Dairy (Butter, Milk Solids)",
    "Dairy (Cheese Powder)", "Dairy (Cheese Powder, Milk Solids)", "Dairy (Cheese)", 
    "Dairy (Cheese, Milk Solids)", "Dairy (Cream, Milk, Butter)", "Dairy (Cream, Milk, Whey)",
    "Dairy (Ghee)", "Dairy (Ghee, Milk Solids)", "Dairy (Goat Milk)", "Dairy (Milk Fat)",
    "Dairy (Milk Powder)", "Dairy (Milk Powder, Lactose)", "Dairy (Milk Powder, Whey)",
    "Dairy (Milk Protein)", "Dairy (Milk Solids)", "Dairy (Milk Solids, Butter)", 
    "Dairy (Milk Solids, Cheese)", "Dairy (Milk Solids, Ghee)", "Dairy (Milk)", 
    "Dairy (Milk, Cream)", "Dairy (Milk, Lactose, Butterfat, Yogurt)", 
    "Dairy (Paneer, Cream, Butter)", "Dairy (Paneer, Yogurt)", "Dairy (Whey)", 
    "Dairy (Whey, Caseinate, Milk Solids)", "Dairy (Whey, Milk Protein)", 
    "Dairy (Whey, Milk)", "Dairy (Yogurt)", "Dairy (Yogurt, Milk Solids)", 
    "Dairy (if Ghee is used)", "Egg", "Fish", "GLUTEN", "Gluten (Barley Malt)", 
    "Gluten (Barley)", "Gluten (Barley, Wheat)", "Gluten (Breadcrumbs)", "Gluten (Oats)", 
    "Gluten (Oats, Wheat)", "Gluten (Oats, Wheat, Barley)", "Gluten (Semolina)", 
    "Gluten (Wheat)", "Gluten (Wheat, Barley Malt)", "Gluten (Wheat, Barley)", 
    "Gluten (Wheat, Barley, Oats)", "Gluten (Wheat, Oat, Barley)", "Gluten (Wheat, Oat, Spelt)",
    "Gluten (Wheat, Oats, Barley)", "Gluten (Wheat, Semolina)", "Gluten (Wheat/Semolina)", 
    "MILK", "MUSTARD", "May contain Dairy", "May contain Gluten", 
    "May contain Gluten, Egg, other Nuts", "May contain Gluten, Milk, Soy", 
    "May contain Gluten, Soy, Milk", "May contain Gluten, Soya", 
    "May contain Milk, Soy, Wheat, Egg, Peanut, Tree Nut, Fish, Shellfish", 
    "May contain Nuts", "May contain Nuts, Seeds", "May contain Peanuts", 
    "May contain Soy", "May contain Soy, Milk", "May contain Soy, Nuts, Peanuts", 
    "May contain Soya", "May contain Soya, Milk, Nuts", "May contain Soya, Wheat", 
    "May contain Soya, Wheat, Milk", "May contain Soya/Peanut depending on type", 
    "May contain Wheat, Soy, Milk", "May contain nuts", "May contain nuts if Badam flavour", 
    "May contain other nuts", "May contain traces of Milk and Soy", 
    "May contain traces of Soya and Nuts", "May contain traces of nuts", 
    "May contain traces of shellfish", "Mustard", "NUTS", "OATS", "Peanuts", 
    "Phenylalanine", "RAGI", "SESAME", "SOY", "SULPHITE", "Sesame", 
    "Sesame (Tahini)", "Soya", "Soya (Emulsifier)", "Soya (Lecithin)", 
    "Soya (Lecithins)", "Soya (Soy Lecithin)", "Soya (Soybean oil)", 
    "Soya (in chocolate compound)", "Sulphites (in Tutti Frutti)", 
    "Sulphites (in candied fruit)", "Sulphites (in dried fruits)", "nuts", 
    "nuts (Almonds)", "nuts (Almonds, Cashews)", "nuts (Almonds, Cashews, Peanuts)", 
    "nuts (Almonds, Cashews, Pistachios)", "nuts (Almonds, Coconut)", 
    "nuts (Almonds, Peanuts)", "nuts (Almonds, Pistachios)", "nuts (Almonds, Raisins)", 
    "nuts (Cashew)", "nuts (Cashew, Almond, Raisin)", "nuts (Cashews)", 
    "nuts (Cashews, Almonds)", "nuts (Cashews, Almonds, Pistachios)", 
    "nuts (Groundnut)", "nuts (Hazelnuts)", "nuts (Peanuts)", "nuts (Peanuts, Almonds)", 
    "nuts (Peanuts, Cashews)", "nuts (Pistachios, Almonds)", "nuts (if added)"
]

In [7]:
# Group entries
grouped_allergies = defaultdict(list)
ungrouped = []

for entry in allergy_entries:
    entry_lower = entry.lower()
    found_group = False
    for group, keywords in category_keywords.items():
        if any(keyword in entry_lower for keyword in keywords):
            grouped_allergies[group].append(entry)
            found_group = True
            break
    if not found_group:
        ungrouped.append(entry)

# Output grouped results
print("\nGrouped Allergy Categories:\n")
for group, items in grouped_allergies.items():
    print(f"== {group} ({len(items)} entries) ==")
    for i in sorted(set(items)):
        print("-", i)
    print()

if ungrouped:
    print("== Uncategorized Entries ==")
    for i in sorted(set(ungrouped)):
        print("-", i)


Grouped Allergy Categories:

== Gluten (24 entries) ==
- BARLEY
- GLUTEN
- Gluten (Barley Malt)
- Gluten (Barley)
- Gluten (Barley, Wheat)
- Gluten (Breadcrumbs)
- Gluten (Oats)
- Gluten (Oats, Wheat)
- Gluten (Oats, Wheat, Barley)
- Gluten (Semolina)
- Gluten (Wheat)
- Gluten (Wheat, Barley Malt)
- Gluten (Wheat, Barley)
- Gluten (Wheat, Barley, Oats)
- Gluten (Wheat, Oat, Barley)
- Gluten (Wheat, Oat, Spelt)
- Gluten (Wheat, Oats, Barley)
- Gluten (Wheat, Semolina)
- Gluten (Wheat/Semolina)
- May contain Gluten
- May contain Gluten, Egg, other Nuts
- May contain Gluten, Soya
- May contain Soya, Wheat
- OATS

== Celery (2 entries) ==
- CELERY
- Celery

== Nuts (35 entries) ==
- Coconut
- Coconut (if used)
- May contain Nuts
- May contain Nuts, Seeds
- May contain Peanuts
- May contain Soy, Nuts, Peanuts
- May contain Soya/Peanut depending on type
- May contain nuts
- May contain nuts if Badam flavour
- May contain other nuts
- May contain traces of Soya and Nuts
- May contain traces 

## REGULAR EXPRESSION

In [9]:
allergy_groups ={
  "Dairy": [
    "Dairy (Butter)",
    "Dairy (Butter, Cream)",
    "Dairy (Butter, Milk Solids)",
    "Dairy (Cheese Powder)",
    "Dairy (Cheese Powder, Milk Solids)",
    "Dairy (Cheese)",
    "Dairy (Cheese, Milk Solids)",
    "Dairy (Cream, Milk, Butter)",
    "Dairy (Cream, Milk, Whey)",
    "Dairy (Ghee)",
    "Dairy (Ghee, Milk Solids)",
    "Dairy (Goat Milk)",
    "Dairy (Milk Fat)",
    "Dairy (Milk Powder)",
    "Dairy (Milk Powder, Lactose)",
    "Dairy (Milk Powder, Whey)",
    "Dairy (Milk Protein)",
    "Dairy (Milk Solids)",
    "Dairy (Milk Solids, Butter)",
    "Dairy (Milk Solids, Cheese)",
    "Dairy (Milk Solids, Ghee)",
    "Dairy (Milk)",
    "Dairy (Milk, Cream)",
    "Dairy (Milk, Lactose, Butterfat, Yogurt)",
    "Dairy (Paneer, Cream, Butter)",
    "Dairy (Paneer, Yogurt)",
    "Dairy (Whey)",
    "Dairy (Whey, Caseinate, Milk Solids)",
    "Dairy (Whey, Milk Protein)",
    "Dairy (Whey, Milk)",
    "Dairy (Yogurt)",
    "Dairy (Yogurt, Milk Solids)",
    "Dairy (if Ghee is used)",
    "May contain Dairy",
    "May contain traces of Milk and Soy"
  ],
  "Gluten": [
    "GLUTEN",
    "Gluten (Barley Malt)",
    "Gluten (Barley)",
    "Gluten (Barley, Wheat)",
    "Gluten (Breadcrumbs)",
    "Gluten (Oats)",
    "Gluten (Oats, Wheat)",
    "Gluten (Oats, Wheat, Barley)",
    "Gluten (Semolina)",
    "Gluten (Wheat)",
    "Gluten (Wheat, Barley Malt)",
    "Gluten (Wheat, Barley)",
    "Gluten (Wheat, Barley, Oats)",
    "Gluten (Wheat, Oat, Barley)",
    "Gluten (Wheat, Oat, Spelt)",
    "Gluten (Wheat, Oats, Barley)",
    "Gluten (Wheat, Semolina)",
    "Gluten (Wheat/Semolina)",
    "May contain Gluten",
    "May contain Gluten, Egg, other Nuts",
    "May contain Gluten, Milk, Soy",
    "May contain Gluten, Soy, Milk",
    "May contain Gluten, Soya"
  ],
  "Nuts": [
    "NUTS",
    "May contain Nuts",
    "May contain Nuts, Seeds",
    "May contain Peanuts",
    "May contain nuts",
    "May contain nuts if Badam flavour",
    "May contain other nuts",
    "May contain traces of nuts",
    "nuts",
    "nuts (Almonds)",
    "nuts (Almonds, Cashews)",
    "nuts (Almonds, Cashews, Peanuts)",
    "nuts (Almonds, Cashews, Pistachios)",
    "nuts (Almonds, Coconut)",
    "nuts (Almonds, Peanuts)",
    "nuts (Almonds, Pistachios)",
    "nuts (Almonds, Raisins)",
    "nuts (Cashew)",
    "nuts (Cashew, Almond, Raisin)",
    "nuts (Cashews)",
    "nuts (Cashews, Almonds)",
    "nuts (Cashews, Almonds, Pistachios)",
    "nuts (Groundnut)",
    "nuts (Hazelnuts)",
    "nuts (Peanuts)",
    "nuts (Peanuts, Almonds)",
    "nuts (Peanuts, Cashews)",
    "nuts (Pistachios, Almonds)",
    "nuts (if added)",
    "May contain Soy, Nuts, Peanuts",
    "May contain Soya, Milk, Nuts",
    "May contain traces of Soya and Nuts"
  ],
  "Soya": [
    "SOY",
    "Soya",
    "Soya (Emulsifier)",
    "Soya (Lecithin)",
    "Soya (Lecithins)",
    "Soya (Soy Lecithin)",
    "Soya (Soybean oil)",
    "Soya (in chocolate compound)",
    "May contain Soy",
    "May contain Soy, Milk",
    "May contain Soya",
    "May contain Soya, Wheat",
    "May contain Soya, Wheat, Milk",
    "May contain Soya/Peanut depending on type"
  ],
  "Egg": [
    "Egg",
    "May contain Gluten, Egg, other Nuts",
    "May contain Milk, Soy, Wheat, Egg, Peanut, Tree Nut, Fish, Shellfish"
  ],
  "Fish and Shellfish": [
    "Fish",
    "Crustaceans (Crab)",
    "Crustaceans (Prawns)",
    "May contain Milk, Soy, Wheat, Egg, Peanut, Tree Nut, Fish, Shellfish",
    "May contain traces of shellfish"
  ],
  "Mustard": [
    "MUSTARD",
    "Mustard"
  ],
  "Sesame": [
    "SESAME",
    "Sesame",
    "Sesame (Tahini)"
  ],
  "Sulphites": [
    "SULPHITE",
    "Sulphites (in Tutti Frutti)",
    "Sulphites (in candied fruit)",
    "Sulphites (in dried fruits)"
  ],
  "Others": [
    "BARLEY",
    "CELERY",
    "Celery",
    "Coconut",
    "Coconut (if used)",
    "May contain Milk, Soy, Wheat, Egg, Peanut, Tree Nut, Fish, Shellfish",
    "May contain traces of Milk and Soy",
    "OATS",
    "Peanuts",
    "Phenylalanine",
    "RAGI"
  ]
}


In [11]:
import re

regex_map = {}
for category, phrases in allergy_groups.items():
    for phrase in phrases:
        # Escape phrase for regex, but allow wildcards for generalization if needed
        pattern = re.escape(phrase.strip()).replace(r'\(', r'\(').replace(r'\)', r'\)')
        regex_map[pattern] = category

# Convert to a list of (compiled_regex, category) for efficient matching
compiled_regexes = [(re.compile(rf"^{pattern}$", re.IGNORECASE), category) for pattern, category in regex_map.items()]


In [12]:
# Sample raw allergy list to convert
raw_allergy_list = [
    "Dairy (Milk Solids)", 
    "Gluten (Wheat)", 
    "nuts (Almonds)", 
    "Soya (Lecithin)", 
    "May contain Soya", 
    "Egg", 
    "Crustaceans (Prawns)"
]

# Function to normalize entries to parent categories
def normalize_allergies(allergy_list):
    normalized = []
    for entry in allergy_list:
        matched = False
        for regex, category in compiled_regexes:
            if regex.match(entry):
                normalized.append(category)
                matched = True
                break
        if not matched:
            normalized.append("Unknown")  # or keep entry as-is
    return sorted(set(normalized))

# Run normalization
normalized_allergies = normalize_allergies(raw_allergy_list)


In [13]:

# Output
print(json.dumps(normalized_allergies, indent=2))

[
  "Dairy",
  "Egg",
  "Fish and Shellfish",
  "Gluten",
  "Nuts",
  "Soya"
]
